In [1]:
import numpy as np
import math

In [2]:
# variables
T_c = 0.00000009775
f_L1 = 1575420000
N_i = 4
CN0 = 15
B_pll = 0.5
B_fe = 26000000
T = 0.020
d = 0.3
pi = math.pi
c = 299792458
B_dll = 0.5
delta_t = 60
sigma_uere = 0 # can't find a value for this

In [3]:
h_0 = 1.4*(10**(-22))
h_m1 = 2.3*(10**(-26))
h_m2 = 3.3*(10**(-31))

dt = 60

pi = math.pi

e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*((pi*h_m2*dt)**2)*dt
e2 = h_m1*dt + h_m2*(pi*dt)**2
e3 = (h_0/(2*dt))+ 4*h_m1 + (8/3)*(pi*pi)*h_m2*dt

Q = np.array([[e1,e2],[e2,e3]])
print(Q)

[[4.36560000e-21 1.39172509e-24]
 [1.39172509e-24 1.25918778e-24]]


In [4]:
s_dll = (B_dll/(2*CN0))*(1/(T_c*B_fe))*(1+(1/(T*CN0)))
pr_var = ((c*T_c)**2)*s_dll

s_pll = (B_pll/(CN0))*(1+(1/(2*T*CN0)))
prr_var = ((c**2)/((2*pi*f_L1 * T_c)**2)) * (s_pll)


#2 satellites
R = np.array([[pr_var,0,0,0],[0,pr_var,0,0],[0,0,prr_var,0],[0,0,0,prr_var]])

print(R)

[[2.44036996e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.44036996e+01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.53302279e+09 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.53302279e+09]]


In [5]:
def time_update(x, P):
  # time update steps
  A = np.array([[1, delta_t], [0, 1]])
  x = A @ x
  P = A @ P @ A.T + Q

  return x, P

In [6]:
def incremental_update(pseudorange_received, pseudorange_expected, pseudorange_rate_received, pseudorange_rate_expected, x, P, N):
  # Calculate pseudorange/range-rate residuals using Eqs. (23)-(25), add in speed of light
  def calculate_residuals(pseudorange_received, pseudorange_expected, pseudorange_rate_received, pseudorange_rate_expected):
    pseudorange_residuals = pseudorange_received - pseudorange_expected
    pseudorange_rate_residuals = pseudorange_rate_received - pseudorange_rate_expected
    z = np.concatenate((pseudorange_residuals, pseudorange_rate_residuals))
    return z

  z = calculate_residuals(pseudorange_received, pseudorange_expected, pseudorange_rate_received, pseudorange_rate_expected)

  # construct C
  C_list = []
  for n_i in N:
    C_i = np.block([[np.ones((n_i, 1)), np.zeros((n_i, 1))], [np.zeros((n_i, 1)), np.ones((n_i, 1))]])
    C_list.append(C_i)
  C = np.vstack(C_list)

  # Calculate measurement noise covariance matrix using Eqs. (29)-(33)
  s_dll = (B_dll/(2*CN0))*(1/(T_c*B_fe))*(1+(1/(T*CN0)))
  sigma_pseudorange = (C * T_c) ** 2 * s_dll + sigma_uere
  sigma_chipspersec = (B_pll / CN0) * (1 + 1/(2 * T * CN0))
  sigma_pseudorange_rate = (c ** 2 / (4 * (pi ** 2) * (f_L1 ** 2))) * (1 / (T_c **2)) * sigma_chipspersec

  R = np.diag(np.concatenate((sigma_pseudorange, sigma_pseudorange_rate))) # returns the measurement noise covariance matrix

  # Update the state and covariance estimate with Eqs. (26)-(28)
  def update_state_covariance(P, C, z, x, R):
    K_n = P @ C.T @ np.linalg.inv(C @ P @ C.T + R) # Kalman gain matrix; R defined earlier
    x_hat_n = x + K_n @ (z - C @ x)
    P_hat_n = (np.eye(len(x)) - K_n @ C) @ P
    return x_hat_n, P_hat_n


  x_hat_n, P_hat_n = update_state_covariance(P, C, z, x, R)
  return x_hat_n, P_hat_n, z

In [7]:
# e_Aj is the error in virtual global clock estimate.
# T_gRN is the global reference clock time of the LSCN at any time instant
# T_gRA is the true global reference time
# T_lB_i is the local time at base station i
# T_l_n is the local time at time instance t(n)
# d_hat_ij is average of the additional delay
# r_ij is the range
# b_hat_A_i is the estimate of the clock bias

def crosslink_update(x_hat_n, P_hat_n, T_ln_i, T_ln1_i, T_lB_i, T_gRA, r_ij, T_gRn, T_gRn1, b_hat_A_i, neighbors):
  H = np.array([1, 0])
  psi_n = np.copy(x_hat_n)

  for j in neighbors:
    b_n_i = T_ln_i - T_gRn # Equation 3, for time step n
    b_n1_i = T_ln1_i - T_gRn1 # Equation 3, for time step n-1
    b_dothat_n_1 = (b_n_i - b_n1_i) / (T_ln_i - T_ln1_i) # clock drift estimate

    d_hat_ij = 0 # average additional delay
    D_hat_ij = (r_ij / c) + d_hat_ij # Equation 13

    b_A_j = T_lA_j - T_gRn - b_hat_A_i # Equation 3
    e_Aj = b_A_j - b_hat_A_i # Equation 4
    T_hat_gA_j = T_gRA + e_Aj # Equation 12
    z_jB_i = c * (T_lB_i - T_hat_gA_j - D_hat_ij) # Equation 16
    z_jn_i = z_jB_i + c * (b_dothat_n_1) * (T_ln_i - T_lB_i) # Equation 34

    d_tilde = np.random.normal(0, 1)  # zero mean random term
    R_jn_i = d_tilde + P_hat_n[0, 0]

    K_ij_n = P_hat_n @ H.T @ np.linalg.inv(H @ P_hat_n @ H.T + R_jn_i)
    psi_n = psi_n + K_ij_n @ (z_jn_i - H @ psi_n)
    P_hat_n = (np.eye(len(psi_n)) - K_ij_n @ H) @ P_hat_n

  return psi_n, P_hat_n

In [8]:
def diffusion_update(P_hat_n, neighbors, beta, T_l_n):
  T_g_sum = 0
  for j in neighbors:
    T_hat_g_j = T_l_n[j] - (1 / c) * P_hat_n[j][0]
    T_g_sum += beta[j] * T_hat_g_j # Equation 40

  T_hat_g_n = T_g_sum

  # equation 41
  b_hat_n = T_l_n - T_hat_g_n

  return b_hat_n

In [9]:
def kalman_filer_loop(M, x_initial, P_initial, neighbors, pseudorange_received, pseudorange_expected, pseudorange_rate_received, pseudorange_rate_expected, T_ln, T_ln1, T_lB, T_gRA, r, T_gRn, T_gRn1, b_hat_A, N):
  x = x_initial
  P = P_initial

  for i in range(1, M):
    # time update step
    x, P = time_update(x, P)

    # incremental update step
    x, P, z = incremental_update(pseudorange_received, pseudorange_expected, pseudorange_rate_received, pseudorange_rate_expected, x, P, N)

    # initialize psi_n with x from step 2 (can choose from step 1 or 2)
    psi_n = x

    # crosslink update step
    psi_n, P = crosslink_update(x, P, T_ln[i], T_l(n-1)[i], T_lB[i], T_gRA, r[i], T_gRn, T_gR(n-1), b_hat_A[i], neighbors)

    # diffusion update step
    b_hat_n = diffusion_update(P, neighbors, beta, T_l_i)

    # phi values become x values
    x = psi_n
    P = P

  return x, P